In [ ]:
# %pip install neo4j
# %pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
from neo4j import GraphDatabase, basic_auth
import os

In [ ]:
# setting the environment
if os.path.exists('ws.env'):
    load_dotenv('ws.env', override=True)

    # Neo4j
    HOST = os.getenv('HOST')
    USERNAME = os.getenv('USERNAME')
    PASSWORD = os.getenv('PASSWORD')
    DATABASE = os.getenv('DATABASE')

print("Environment loaded")

In [ ]:
# checking the connection
AUTH = (USERNAME, PASSWORD)

with GraphDatabase.driver(HOST, auth=AUTH) as driver:
    try:
        driver.verify_connectivity()
        print("Connection verified.")
    except Exception as e:
        print(e)

In [ ]:
# create the grid
AUTH = (USERNAME, PASSWORD)

def countthenodes(tx):
    result = tx.run("""
        MATCH (:Cell)
        RETURN count(*) AS thecount
        """)
    return result.single(strict=True)

def counttherelationships(tx):
    result = tx.run("""
        RETURN 
          COUNT {MATCH (:Cell)-[:ROW]->(:Cell)} AS theROWcount,
          COUNT {MATCH (:Cell)-[:COLUMN]->(:Cell)} AS theCOLUMNcount,
          COUNT {MATCH (:Cell)-[:BOX]->(:Cell)} AS theBOXcount
        """)
    return result.single(strict=True)

def createthenodes(tx):
    result = tx.run("""
        FOREACH(row IN range(1,9) |
          FOREACH(column IN range(1,9) |
            CREATE (:Cell {
              id: toInteger(toString(row) + toString(column)), 
              stringid: "r" + toString(row) + "c" + toString(column),
              row: row,
              column: column,
              gridlocation: point({ x: column - 1, y: (row - 9) * -1 }),
              value: 0,
              box: 3 * ( ( row  - 1 ) / 3 ) + ( ( column  - 1 ) / 3 ) + 1
            })
          )
        )
        WITH "created the nodes" AS result
        MATCH (:Cell)
        RETURN count(*) AS thecount
        """)
    return result.single(strict=True)

def createtherows(tx):
    result = tx.run("""
        MATCH (c:Cell)
        WITH c.row AS row, c.id AS id
        ORDER BY row, id
        WITH row, collect(id) AS toconnect
        CALL (row, toconnect) {
          UNWIND range(0,8) AS index
          WITH toconnect[index] AS sourceid, toconnect[index+1] AS targetid
          MATCH (source:Cell WHERE source.id = sourceid)
          MATCH (target:Cell WHERE target.id = targetid)
          CREATE (source)-[:ROW]->(target)
          RETURN count(*) AS connections
        }
        RETURN sum(connections) AS thecount
        """)
    return result.single(strict=True) 

def createthecolumns(tx):
    result = tx.run("""
        MATCH (c:Cell)
        WITH c.column AS column, c.id AS id
        ORDER BY column, id
        WITH column, collect(id) AS toconnect
        CALL (column, toconnect) {
          UNWIND range(0,8) AS index
          WITH toconnect[index] AS sourceid, toconnect[index+1] AS targetid
          MATCH (source:Cell WHERE source.id = sourceid)
          MATCH (target:Cell WHERE target.id = targetid)
          CREATE (source)-[:COLUMN]->(target)
          RETURN count(*) AS connections
        }
        RETURN sum(connections) AS thecount    
        """)
    return result.single(strict=True)

def createtheboxes(tx):
    result = tx.run("""
        MATCH (c:Cell)
        WITH c.box AS box, c.id AS id
        ORDER BY box, id
        WITH box, collect(id) AS toconnect
        CALL (box, toconnect) {
          UNWIND range(0,8) AS index
          WITH toconnect[index] AS sourceid, toconnect[index+1] AS targetid
          MATCH (source:Cell WHERE source.id = sourceid)
          MATCH (target:Cell WHERE target.id = targetid)
          CREATE (source)-[:BOX]->(target)
          RETURN count(*) AS connections
        }
        RETURN sum(connections) AS thecount
        """)
    return result.single(strict=True)

with GraphDatabase.driver(HOST, auth=AUTH) as driver:
    try:
        with driver.session(database=DATABASE) as session:
            try:
                numberofnodes = session.execute_read(countthenodes)
                if numberofnodes["thecount"] == 0:
                    numberofnodes = session.execute_write(createthenodes)
                    numberofrowrelationships = session.execute_write(createtherows)
                    numberofcolumnrelationships = session.execute_write(createthecolumns)
                    numberofboxrelationships = session.execute_write(createtheboxes)
                    print("created nodes               : " + str(numberofnodes["thecount"]))
                    print("created ROW relationships   : " + str(numberofrowrelationships["thecount"]))
                    print("created COLUMN relationships: " + str(numberofcolumnrelationships["thecount"]))
                    print("created BOX relationships   : " + str(numberofboxrelationships["thecount"]))                    
                if numberofnodes["thecount"] == 81:
                    numberofrelationships = session.execute_read(counttherelationships)
                    if (numberofrelationships["theROWcount"] == 72 and 
                    numberofrelationships["theCOLUMNcount"] == 72 and 
                    numberofrelationships["theBOXcount"] == 72):
                        print("the grid is complete")
                    else:
                        raise Exception("There should be 72 of each relationship!") 
                else:
                    raise Exception("There should be 81 nodes!") 
            except Exception as sessionerror:
                print(sessionerror)
    except Exception as drivererror:
        print(drivererror)

In [ ]:
# load the puzzle
# replace with your own input as needed
PUZZLE = "020030000003000040004908003040710056300000002510096080900305400080000100000020090"
def setthepuzzle(tx, PUZZLE):
    result = tx.run("""
        WITH split($PUZZLE,'') AS entries
        UNWIND range(1,9) AS row
        UNWIND range(1,9) AS column
        MATCH (cell:Cell WHERE cell.id = toInteger(toString(row) + toString(column)))
        //WHERE cell.value <> toInteger(entries[((row - 1) * 9) + (column - 1)])
        SET cell.value = toInteger(entries[((row - 1) * 9) + (column - 1)])
        """, PUZZLE = PUZZLE)
    return result.consume()

with GraphDatabase.driver(HOST, auth=AUTH) as driver:
    try:
        with driver.session(database=DATABASE) as session:
            try:
                resultsummary = session.execute_write(setthepuzzle, PUZZLE)
                print(str(resultsummary.counters.properties_set) + " properties were set")
            except Exception as sessionerror:
                print(sessionerror)
    except Exception as drivererror:
        print(drivererror)

In [ ]:
# solve with feedback - some query templates

# check if cell is empty and candidate for a certain value
BASEQUERY_value = """
    MATCH (cell:Cell) 
    // the cell is empty
    WHERE cell.value = 0
    // and there's nothing in the cell's row that has the value
    AND NOT EXISTS {
       MATCH (cell)-[:ROW]-+(other) WHERE other.value = value
    }
    // and there's nothing in the cell's column that has the value
    AND NOT EXISTS {
       MATCH (cell)-[:COLUMN]-+(other) WHERE other.value = value
    }
    // and there's nothing in the cell's box that has the value
    AND NOT EXISTS {
      MATCH (cell)-[:BOX]-+(other) WHERE other.value = value
    }
    """

# part of singles queries that's always the same
BASEQUERY_singles = """
    UNWIND range(1,9) AS value
    CALL (value) {
      CALL (value) {
    """ + BASEQUERY_value + """
        WITH cell.box AS box, collect(cell.id) AS theids
        WHERE size(theids) = 1
        // there's only one possibility for the value in the box
        RETURN theids[0] AS single, "box" AS reason
      }
      RETURN single, reason
      UNION
      CALL (value) { 
    """ + BASEQUERY_value + """
        WITH cell.row AS row, collect(cell.id) AS theids
        WHERE size(theids) = 1
        // there's only one possibility for the value in the row
        RETURN theids[0] AS single, "row" AS reason
      }
      RETURN single, reason
      UNION
      CALL (value) {
    """ + BASEQUERY_value + """
        WITH cell.column AS column, collect(cell.id) AS theids
        WHERE size(theids) = 1
        // there's only one possibility for the value in the column
        RETURN theids[0] AS single, "column" AS reason
      }
      RETURN single, reason
    }"""

# part of eliminations queries that's always the same
BASEQUERY_eliminations = """
    UNWIND range(1,9) AS value
    CALL (value) {
      CALL (value) {
    """ + BASEQUERY_value + """
        WITH cell.box AS box, collect(cell.id) AS theids
        RETURN theids AS options, "box" AS reason 
      }
      RETURN options, reason
      UNION
      CALL (value) { 
    """ + BASEQUERY_value + """
        WITH cell.row AS row, collect(cell.id) AS theids
        RETURN theids AS options, "row" AS reason
      }
      RETURN options, reason
      UNION
      CALL (value) {
    """ + BASEQUERY_value + """
        WITH cell.column AS column, collect(cell.id) AS theids
        RETURN theids AS options, "column" AS reason
      }
      RETURN options, reason
    }
    UNWIND options AS option
    WITH option, collect(value) as values
    WITH option, reduce(uniquevalues=[], value in values | CASE WHEN value IN uniquevalues THEN uniquevalues     ELSE uniquevalues + value END) AS uniquevalues
    WHERE size(uniquevalues) = 1
    """

In [ ]:
# solve with feedback - the functions

def lefttodo(tx):
    result = tx.run("""
        MATCH (c:Cell WHERE c.value = 0)
        RETURN count(*) AS todo
        """)
    return result.single(strict=True)

def findnakedsinglescount(tx):
    result = tx.run(BASEQUERY_singles + 
        """
        WITH value, single, collect(reason) AS reasons
        ORDER BY value, single
        RETURN count(*) AS thecount
        """)
    return result.single(strict=True)

def findeliminationscount(tx):
    result = tx.run(BASEQUERY_eliminations + 
        """
        WITH option, uniquevalues[0] AS single
        RETURN count(*) AS thecount
        """)
    return result.single(strict=True)

def findnakedsingles(tx):
    result = tx.run(BASEQUERY_singles + 
        """
        RETURN value, single, collect(reason) AS reasons
        ORDER BY value, single        
        """)
    return list(result)

def findeliminations(tx):
    result = tx.run(BASEQUERY_eliminations + 
        """
        RETURN option, uniquevalues[0] AS single;
        """)
    return list(result)
    
def fillnakedsingles(tx):
    result = tx.run(BASEQUERY_singles + 
        """
        WITH value, single, collect(reason) as reasons
        MATCH (cell:Cell WHERE cell.id = single)
        SET cell.value = value
        RETURN count(*) AS thecount
        """)
    return result.single(strict=True)

def filleliminations(tx):
    result = tx.run(BASEQUERY_eliminations + 
        """
        MATCH (cell:Cell WHERE cell.id = option)
        SET cell.value = uniquevalues[0]
        RETURN count(*) AS thecount
        """)
    return result.single(strict=True)

In [ ]:
# solve with feedback - the solve

with GraphDatabase.driver(HOST, auth=AUTH) as driver:
    try:
        with driver.session(database=DATABASE) as session:
            try:
                print("START solve")
                left = session.execute_read(lefttodo)
                iteration = 1
                while left["todo"] > 0:
                    print("Iteration - " + str(iteration)) 
                    save = left["todo"]
                    print("1. Find naked singles")
                    nakedsingles = session.execute_read(findnakedsinglescount)
                    if nakedsingles["thecount"] > 0:
                        print("   found", nakedsingles["thecount"])
                        nsresult = session.execute_read(findnakedsingles)
                        for nakedsingle in nsresult:
                            print("  ", nakedsingle["single"], nakedsingle["value"],nakedsingle["reasons"])
                        updatednakedsingles = session.execute_write(fillnakedsingles)
                        print("   updated", updatednakedsingles["thecount"])
                    else:
                        print("2. Find eliminations")
                        eliminations = session.execute_read(findeliminationscount)
                        if eliminations["thecount"] > 0:
                            print("   found", eliminations["thecount"])
                            eresult = session.execute_read(findeliminations)
                            for elimination in eresult:
                                print("  ", elimination["option"], elimination["single"])
                            updatedeliminations = session.execute_write(filleliminations)
                            print("   updated", updatedeliminations["thecount"])
                            
                    print("X. Check what is left todo")
                    left = session.execute_read(lefttodo)
                    if left["todo"] == save:
                        print("nothing changed, lefttodo is still " + str(left["todo"]))
                        break
                    else:
                        print("   left", left["todo"])
                    iteration = iteration + 1
                print("END solve")
            except Exception as sessionerror:
                print(sessionerror)
    except Exception as drivererror:
        print(drivererror)